In [10]:
import tensorflow as tf
import importlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
import utils_triplet2
import model2
import results_generator
importlib.reload(utils_triplet2)
importlib.reload(model2)
importlib.reload(results_generator)
from utils_triplet2 import *
from model2 import *
from results_generator import *
tf.test.is_built_with_cuda()

True

In [2]:
# This code is only needed if the dataset is stored in google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Check if gpu exists
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [3]:
# This code is only needed if the dataset is stored in google drive and it is 7z compressed
!p7zip -d /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/MyDrive/Thesis/                                         1 file, 263683120 bytes (252 MiB)

Extracting archive: /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z
--
Path = /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z
Type = 7z
Physical Size = 263683120
Headers Size = 344455
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 692 - Sketch-Icon-Dataset/icon/emoji/tired1.jpg                                                      1% 1166 - Sketch-Icon-Dataset/icon/social/share.jpg

In [11]:
icon_name_category, sketch_name_category = get_icons_and_sketches()

In [12]:
# both icon categories and sketch categories have the same order when we are reading them
# as a result the categories have the same one hot encoding in both dictionaries
icon_categories =  np.unique(icon_name_category[:, 1], return_index=False)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(icon_categories)

# binary encode the classes of icons
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
icon_categories_dic = {}
for i in range(len(icon_categories)):
    icon_categories_dic[icon_categories[i]] = onehot_encoded[i]

In [13]:
sketch_categories =  np.unique(sketch_name_category[:, 1], return_index=False)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(sketch_categories)

# binary encode the classes of sketches
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
sketch_categories_dic = {}
for i in range(len(sketch_categories)):
    sketch_categories_dic[sketch_categories[i]] = onehot_encoded[i]

In [14]:
# Split the dataset into train and test with randomness
# or load the train and test dataset which is created as numpy arrays.
# The purpose of the predefined datasets is to have the same datasets for training
# the classification of sketches and the sketch-icon retrieval, so the test set will be the same,
# and unknown for both trainings
random_generation = False
if random_generation:
    positive_pairs = positive_pairs_generator(icon_name_category, sketch_name_category)
    positive_pairs = shuffle(positive_pairs)

    len_data = len(positive_pairs)
    p_train=0.9
    p_test=0.1
    num_train = int(np.ceil(len_data*p_train))
    num_test = int(np.floor(len_data*p_test))

    positive_pairs_Train = positive_pairs[:num_train]
    possible_negative_pairs_Train = possible_negative_pairs_generator(positive_pairs_Train, icon_name_category)
    triplet_pairs_Train = triplets_generator(positive_pairs_Train, icon_name_category, possible_negative_pairs_Train)
    triplet_pairs_Train = shuffle(triplet_pairs_Train)
    positive_pairs_Test = positive_pairs[-num_test:]
    sketches_Test = positive_pairs_Test[:, [0, 1]]

    icons_Test = positive_pairs_Test[:, [2, 3]]
    # create the icons test dataset by removing the dublicates of the array
    _, unique_indices = np.unique(icons_Test[:,0], return_index=True)
    unique_icons_Test = icons_Test[unique_indices]

    print(f'We have {len(triplet_pairs_Train)} samples in the training set.')
    print(f'We have {len(sketches_Test)} sketches in the test set.')
    print(f'We have {len(unique_icons_Test)} unique icons in the test set.')
else:
    sketch_name_category_Train = load_train_set()
    sketch_name_category_Test = load_test_set()

    positive_pairs_Train =  positive_pairs_generator(icon_name_category, sketch_name_category_Train)
    possible_negative_pairs_Train = possible_negative_pairs_generator(positive_pairs_Train, icon_name_category)
    triplet_pairs_Train = triplets_generator(positive_pairs_Train, icon_name_category, possible_negative_pairs_Train)
    triplet_pairs_Train = shuffle(triplet_pairs_Train)

    positive_pairs_Test =  positive_pairs_generator(icon_name_category, sketch_name_category_Test)

    sketches_Test = positive_pairs_Test[:, [0, 1]]

    icons_Test = positive_pairs_Test[:, [2, 3]]
    # create the icons test dataset by removing the dublicates of the array
    _, unique_indices = np.unique(icons_Test[:,0], return_index=True)
    unique_icons_Test = icons_Test[unique_indices]

    print(f'We have {len(triplet_pairs_Train)} samples in the training set.')
    print(f'We have {len(sketches_Test)} sketches in the test set.')
    print(f'We have {len(unique_icons_Test)} unique icons in the test set.')

We have 28626 samples in the training set.
We have 3180 sketches in the test set.
We have 1178 unique icons in the test set.


In [15]:
# Creation of folders for storing stats, weights and the hyperparameters of the model
algorithmName = "Triplet-Classification Loss-CWI" # CWI = Classification Weights Initialization
model_name = "GoogleNet"
create_directory(model_name + "/")
now = datetime.now()
date_time_folder = now.strftime("%d-%m-%Y %H-%M-%S")
train_weights_folder = "Train Weights"

create_directory(model_name + "/" + algorithmName)
current_run_path = model_name + "/" + algorithmName + "/" + date_time_folder + "/"
create_directory(current_run_path)
train_weights_path = model_name + "/" + algorithmName + "/" + date_time_folder + "/" + train_weights_folder
create_directory(train_weights_path)
#=====================================================================================

# Hyperparameter initializations
BATCH_SIZE = 64
num_epochs = 1000
margin = 1
learing_rate = 0.00001

# Store the hyperparameters of the model into a file to remember what I used in a specific run
write_hyperparameters_in_file(current_run_path, learing_rate, BATCH_SIZE, margin)

# Initializations of optimizer, loss and network and set the backend (model variables) to float32
optimizer = tf.keras.optimizers.Adam(learing_rate)
tf.keras.backend.set_floatx('float32')
weights_path_sketch = "/content/gdrive/MyDrive/Thesis/GoogleNet/Sketch Classification/22-03-2021 17-49-36/Train Weights/"
weights_path_icon = "/content/gdrive/MyDrive/Thesis/GoogleNet/Icon Classification/23-03-2021 13-49-21/Train Weights/"
iconClassificationModel = googlenet(len(icon_categories))
iconClassificationModel.load_weights(weights_path_icon + 'iconClassification')
sketchClassificationModel = googlenet(len(sketch_categories))
sketchClassificationModel.load_weights(weights_path_sketch + 'sketchClassification')

loss = triplet_loss
loss2 = compute_cross_entropy

# Function which is responsible to calculate the loss and update the grads
def train_step(sketches, positive_icons, negative_icons, targets_icons, targets_sketches, margin):
  with tf.GradientTape(persistent=True) as tape:
      sketch_outputs = sketchClassificationModel(sketches, training = True)
      sketch_main, sketch_aux1, sketch_aux2, sketch_features =  sketch_outputs[0], sketch_outputs[1], sketch_outputs[2], sketch_outputs[3]
      tape.watch(sketch_main)
      tape.watch(sketch_aux1)
      tape.watch(sketch_aux2)
      tape.watch(sketch_features)
      sketch_loss_main = loss2(sketch_main, targets_sketches)
      sketch_loss_aux1 = loss2(sketch_aux1, targets_sketches)
      sketch_loss_aux2 = loss2(sketch_aux2, targets_sketches)
      sketch_loss = sketch_loss_main + 0.3 * sketch_loss_aux1 + 0.3 * sketch_loss_aux2

      icon_positive_outputs = iconClassificationModel(positive_icons, training = True)
      icon_positive_main, icon_positive_aux1, icon_positive_aux2, icon_positive_features =  icon_positive_outputs[0], icon_positive_outputs[1], icon_positive_outputs[2], icon_positive_outputs[3]
      tape.watch(icon_positive_main)
      tape.watch(icon_positive_aux1)
      tape.watch(icon_positive_aux2)
      tape.watch(icon_positive_features)
      icon_loss_main = loss2(icon_positive_main, targets_icons)
      icon_loss_aux1 = loss2(icon_positive_aux1, targets_icons)
      icon_loss_aux2 = loss2(icon_positive_aux2, targets_icons)
      icon_loss = icon_loss_main + 0.3 * icon_loss_aux1 + 0.3 * icon_loss_aux2

      icon_negative_features = iconClassificationModel(negative_icons, training = True)[3]
      tape.watch(icon_negative_features)

      current_loss = loss(sketch_features, icon_positive_features, icon_negative_features, margin)

  grads = tape.gradient([current_loss, sketch_loss], sketchClassificationModel.trainable_variables)
  optimizer.apply_gradients((grad, var) for (grad, var) in zip(grads, sketchClassificationModel.trainable_variables) if grad is not None)
  grads = tape.gradient([current_loss, icon_loss], iconClassificationModel.trainable_variables)
  optimizer.apply_gradients((grad, var) for (grad, var) in zip(grads, iconClassificationModel.trainable_variables) if grad is not None)
  return current_loss

Creation of the directory  failedGoogleNet/
Creation of the directory  failedGoogleNet/Triplet-Classification Loss-CWI
Successfully created the directory  GoogleNet/Triplet-Classification Loss-CWI/01-04-2021 16-11-52/
Successfully created the directory  GoogleNet/Triplet-Classification Loss-CWI/01-04-2021 16-11-52/Train Weights


In [16]:
top_acc = 0
top_acc1 = 0
top_acc10 = 0
for epoch in range(num_epochs):

  # Training loop throught the training dataset
  epoch_loss_avg = tf.keras.metrics.Mean()
  for i in range(0, len(triplet_pairs_Train), BATCH_SIZE):
      batch_triple_pairs = triplet_pairs_Train[i:i+BATCH_SIZE]
      sketches, positive_icons, negative_icons, icon_targets, sketch_targets = get_batch_triplet_classification_low_ram(batch_triple_pairs, icon_categories_dic, sketch_categories_dic)
      loss_value = train_step(sketches, positive_icons, negative_icons, icon_targets, sketch_targets, margin)
      epoch_loss_avg.update_state(loss_value)
  print("Epoch {:d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))
  if epoch%1==0:
      # Test loop through the test datatet
      acc_1 = 0
      acc_10 = 0

      # exctract sketch features
      sketch_representations = []
      for j in range(0, len(sketches_Test), BATCH_SIZE):
          batch_sketches = sketches_Test[j:j+BATCH_SIZE]
          sketches_array = get_batch_sketches(batch_sketches)
          sketch_repr = sketchClassificationModel(sketches_array, training = False)[3]
          sketch_representations.append(sketch_repr)
      sketch_representations = np.vstack(sketch_representations)

      # exctract icon features
      icon_representations = []
      for j in range(0, len(unique_icons_Test), BATCH_SIZE):
          batch_icons = unique_icons_Test[j:j+BATCH_SIZE]
          icons_array = get_batch_icons(batch_icons)
          icons_repr = iconClassificationModel(icons_array, training = False)[3]
          icon_representations.append(icons_repr)
      icon_representations = np.vstack(icon_representations)

      # check using euclidean distance the top 1 and top 10 accuracy
      for k in range(len(sketch_representations)):
          sketch_repr = sketch_representations[k]
          sketch_representations_tile = np.tile(sketch_repr, len(unique_icons_Test)).reshape(len(unique_icons_Test), 256)
          diff = np.sqrt(np.mean((sketch_representations_tile - icon_representations)**2, -1))
          top_k = np.argsort(diff)[:10]
          
          for j in range(len(top_k)):
              index = top_k[j]
              if j == 0 and sketches_Test[k][0].split("_")[0] == unique_icons_Test[index][0].replace(".jpg",""):
                  acc_1 = acc_1 + 1
                  acc_10 = acc_10 + 1
                  break
              elif sketches_Test[k][0].split("_")[0] == unique_icons_Test[index][0].replace(".jpg",""):
                  acc_10 = acc_10 + 1
                  break
      if top_acc < acc_1 + acc_10:
        save_weights(iconClassificationModel, train_weights_path + "/iconTripletWithCWI")
        save_weights(sketchClassificationModel, train_weights_path + "/sketchTripletWithCWI")
        top_acc = acc_1 + acc_10
        top_acc1 = acc_1
        top_acc10 = acc_10
      print("Accuracy of top 1: " + str(acc_1/len(sketches_Test)))
      print("Accuracy of top 10: " + str(acc_10/len(sketches_Test)))
      write_triplet_stats_in_file(current_run_path, epoch, epoch_loss_avg.result(), acc_1/len(sketches_Test), acc_10/len(sketches_Test))
  
  # generate triplets with different negative pairs and shuffle
  triplet_pairs_Train = triplets_generator(positive_pairs_Train, icon_name_category, possible_negative_pairs_Train)
  triplet_pairs_Train = shuffle(triplet_pairs_Train)
  
print("The best accuracy of top 1: " + str(top_acc1/len(sketches_Test)))
print("The best accuracy of top 10: " + str(top_acc10/len(sketches_Test)))

Epoch 0: Loss: 18263.389
Accuracy of top 1: 0.0009433962264150943
Accuracy of top 10: 0.01509433962264151


KeyboardInterrupt: ignored